In [4]:
from src.newsletters.gmail import Gmail
from src.newsletters.config import NEWSLETTER_AND_PARSER

In [9]:
NEWSLETTER_AND_PARSER.keys()

dict_keys(['TLDR AI <dan@tldrnewsletter.com>'])

In [5]:
gmail = Gmail()

In [10]:
emails = gmail.fetch_emails(sender='TLDR AI <dan@tldrnewsletter.com>')

In [12]:
len(emails)

3

In [13]:
email = emails[0]

In [14]:
email.keys()

dict_keys(['sender', 'subject', 'date_utc', 'id', 'text'])

In [56]:
import re

In [57]:
list_text = email['text'].split('\r\n\r\n')
ii = 0
pattern = r"READ\) \[\d+\]"
while ii < len(list_text):
    matches = re.findall(pattern, list_text[ii])
    if len(matches) > 0:
        print(list_text[ii])
        print(list_text[ii+1])
        ii = ii + 2
    else:
        ii = ii + 1

 ADS MIGHT BE COMING TO CHATGPT (1 MINUTE READ) [6] 
 OpenAI is considering an advertising business model to help support
its costly AI tool development. Despite no current active plans for
ads, the idea remains on the table as a potential revenue source. CEO
Sam Altman considers ads a last resort and has expressed discomfort at
the possibility of combining ads and AI. 
 THE RACE IS ON TO MAKE AI AGENTS DO YOUR ONLINE SHOPPING FOR YOU (8
MINUTE READ) [7] 
 Tech companies are developing AI shopping agents to automate online
purchases, potentially reshaping the retail industry. Perplexity's
current model faces operational challenges. Companies like OpenAI,
Google, and Amazon are also exploring AI purchasing capabilities.
These developments aim to streamline shopping but raise questions
about privacy, retailer interactions, and the evolving online shopping
experience. 
 SALESFORCE CEO MARC BENIOFF HAS THOUGHTS ON AI AGENTS, AUTOMATION,
AND THE FUTURE OF YOUR JOB (28 MINUTE READ) [8] 
 Sal

In [65]:
links = re.finditer(r"\[\d+\] http", email['text'])
list(links)

[<re.Match object; span=(7233, 7241), match='[1] http'>,
 <re.Match object; span=(7277, 7285), match='[2] http'>,
 <re.Match object; span=(7380, 7388), match='[3] http'>,
 <re.Match object; span=(7602, 7610), match='[4] http'>,
 <re.Match object; span=(7674, 7682), match='[5] http'>,
 <re.Match object; span=(7782, 7790), match='[6] http'>,
 <re.Match object; span=(7906, 7914), match='[7] http'>,
 <re.Match object; span=(8029, 8037), match='[8] http'>,
 <re.Match object; span=(8125, 8133), match='[9] http'>,
 <re.Match object; span=(8181, 8190), match='[10] http'>,
 <re.Match object; span=(8238, 8247), match='[11] http'>,
 <re.Match object; span=(8324, 8333), match='[12] http'>,
 <re.Match object; span=(8489, 8498), match='[13] http'>,
 <re.Match object; span=(8556, 8565), match='[14] http'>,
 <re.Match object; span=(8613, 8622), match='[15] http'>,
 <re.Match object; span=(8659, 8668), match='[16] http'>,
 <re.Match object; span=(8736, 8745), match='[17] http'>,
 <re.Match object; span

In [62]:
email['text']

"OpenAI is considering an advertising business model to help support\r\nits costly AI tool development. CEO Sam Altman considers ads a last\r\nresort\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\r\n\r\n\r\n Sign Up [1] |Advertise [2]|View Online [3] \r\n\r\n\t\tTLDR \r\n\r\n\t\tTOGETHER WITH [Augment Code] [4]\r\n\r\nTLDR AI 2024-12-03\r\n\r\n AUGMENT CODE: DEVELOPER AI THAT THINKS IN SYSTEMS, NOT CODE SNIPPETS\r\n(SPONSOR) [4] \r\n\r\n Most AI coding copilots spit out generic code with no context of your\r\ncom